In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate

## Creating the datasets

### Creating the training set

In [2]:
cwd = os.getcwd()
train_file_path = '/'.join(cwd.split('/')[:-2]) + '/preprocessing/data/3_10000/train_3_10000.csv'
train_df = pd.read_csv(train_file_path)

In [3]:
train_df.head()

,label,byte1,byte2,byte3,byte4,byte5,byte6,byte7,byte8,byte9,...,byte1472,byte1473,byte1474,byte1475,byte1476,byte1477,byte1478,byte1479,byte1480,packet_length
0,YouTube,1,187,218,214,118,128,105,228,220,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1428
1,YouTube,1,187,156,22,89,170,242,38,137,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1462
2,YouTube,1,187,236,103,231,92,105,103,177,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1428
3,YouTube,1,187,187,69,162,232,29,4,80,...,129.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1472
4,YouTube,1,187,213,224,91,159,153,121,202,...,183.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1472


In [4]:
train_df = train_df.drop('packet_length', axis = 1)

In [5]:
## converting the labels into unique integers
train_df['label'] = train_df.label.astype('category').cat.codes

### Creating the validation set

In [6]:
validation_file_path = '/'.join(cwd.split('/')[:-2]) + '/preprocessing/data/3_10000/val_3_10000.csv'
validation_df = pd.read_csv(validation_file_path)

In [7]:
validation_df = validation_df.drop('packet_length', axis = 1)

In [8]:
## converting the labels into unique integers
validation_df['label'] = validation_df.label.astype('category').cat.codes

### Creating the testing set

In [9]:
test_file_path = '/'.join(cwd.split('/')[:-2]) + '/preprocessing/data/3_10000/test_3_10000.csv'
test_df = pd.read_csv(test_file_path)

In [10]:
test_df = test_df.drop('packet_length', axis = 1)

In [11]:
## converting the labels into unique integers
test_df['label'] = test_df.label.astype('category').cat.codes

## Training the svc using one vs many

In [12]:
lin_clf = svm.LinearSVC(max_iter=5000, dual=False)

In [13]:
X_train = train_df.drop('label', axis=1)
X_train = X_train / 255.0

In [14]:
y_train = train_df['label']

In [15]:
lin_clf.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=5000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

## Evaluating the models performance on the validation set

In [16]:
X_validation = validation_df.drop('label', axis=1)
X_validation = X_validation / 255.0

In [17]:
y_validation = validation_df['label']

In [18]:
y_pred_validation = lin_clf.predict(X_validation)

In [19]:
print("Validation Accuracy:",accuracy_score(y_validation, y_pred_validation))

Validation Accuracy: 0.8947916666666667


## Evaluating the models performance on the test set

In [20]:
X_test = test_df.drop('label', axis=1)
X_test = X_test / 255.0

In [21]:
y_test = test_df['label']

In [22]:
y_pred_test = lin_clf.predict(X_test)

In [23]:
print("Test Accuracy:",accuracy_score(y_test, y_pred_test))

Test Accuracy: 0.9023333333333333
